In [1]:
# --- Step 1: Import Libraries ---
import pandas as pd
from itertools import combinations
import os
import platform
from math import ceil
import subprocess

In [10]:
# --- Step 2: Define Input & Output Paths ---
current_folder = os.getcwd()
input_file = os.path.join(current_folder, r"C:\Users\Swarupa\Downloads\10 comp 10 days data.csv")
output_folder = os.path.join(current_folder, "Batched_code_data")
os.makedirs(output_folder, exist_ok=True)

In [11]:
# --- Step 3: Read CSV ---
df = pd.read_csv(input_file)
df.columns = df.columns.str.strip()

In [12]:
# --- Step 4: Identify date columns ---
date_columns = [col for col in df.columns if not col.lower().startswith("company") and "quantity" not in col.lower()]
latest_date = date_columns[0]
print(f" Using latest date column: {latest_date}")
print(f" Total date columns found: {len(date_columns)}")

 Using latest date column: 08/19/2025
 Total date columns found: 10


In [13]:
# --- Step 6: Compute constant Quantity per company ---
df["Quantity"] = 50000 / df[latest_date]

In [14]:
# --- Step 7: Prepare lookups ---
company_list = df["Company Name"].tolist()
price_data = df.set_index("Company Name")[date_columns]
quantity_lookup = df.set_index("Company Name")["Quantity"].to_dict()

In [15]:
# --- Step 8: Settings ---
BATCH_SIZE = 110
all_combos = list(combinations(company_list, 4))
total_batches = ceil(len(all_combos) / BATCH_SIZE)
print(f" Total combinations: {len(all_combos)} → {total_batches} batches")

 Total combinations: 210 → 2 batches


In [16]:
# --- Step 8: Process Each Batch ---
for batch_num in range(total_batches):
    start = batch_num * BATCH_SIZE
    end = start + BATCH_SIZE
    batch_combos = all_combos[start:end]
    results = []

    for combo in batch_combos:
        try:
            # Get constant quantities for these 4 companies
            q = [quantity_lookup[c] for c in combo]
            combo_prices = price_data.loc[list(combo)]

            # Total value for each date = Σ(q_i × price_i)
            total_values = (combo_prices.mul(q, axis=0)).sum()

            # --- Compute Backward (future-looking) 50-day moving average ---
            reversed_series = pd.Series(total_values.values[::-1])
            backward_ma = reversed_series.rolling(window=3, min_periods=3).mean()
            ma_series = backward_ma.values[::-1] 

            # Create one row for this combination
            row = {"Combo": ", ".join(combo)}

            # Add Total Value columns horizontally
            for i, date in enumerate(date_columns):
                row[date] = total_values.values[i]

            # Add Backward Moving Average columns horizontally
            for i, date in enumerate(date_columns):
                row[f"MA_{date}"] = ma_series[i]

            results.append(row)
        except KeyError:
            continue  # skip missing data

    # --- Step 9: Save Each Batch to Excel ---
    batch_df = pd.DataFrame(results)
    batch_file = os.path.join(output_folder, f"Batch_{batch_num + 1}.xlsx")
    batch_df.to_excel(batch_file, index=False)
    print(f" Saved: {batch_file}")

 Saved: C:\Users\Swarupa\Batched_code\Batch_1.xlsx
 Saved: C:\Users\Swarupa\Batched_code\Batch_2.xlsx


In [9]:
# --- Step 10: Auto-open Output Folder ---
print("\n All batches processed and saved successfully!")

system_name = platform.system()
try:
    if system_name == "Windows":
        os.startfile(output_folder)
    elif system_name == "Darwin":  # macOS
        subprocess.Popen(["open", output_folder])
    elif system_name == "Linux":
        subprocess.Popen(["xdg-open", output_folder])
    else:
        print(f" Please open manually: {output_folder}")
except Exception as e:
    print(f" Could not open folder automatically: {e}")

print(f" Output folder: {output_folder}")


 All batches processed and saved successfully!
 Output folder: C:\Users\Swarupa\Batched_Output
